# EEG Feature Extraction 1: Frequency and Time Domain Features
In 2008, a BCI Competition was held on EEG datasets to find the best ML and statistical algorithms to differentiate different classes of neural data. The BCI Competition IV 2b is a motor imagery dataset with eye artifact data, making it a very realistic dataset. The subjects are prompted to imagine left vs right hand movement and the EEG + EOG signals for each trial are collected. We here have provided a simpler version of the dataset in CSV format for you to get started with. 

This notebook will help you get a better understanding of the techniques used to preprocess and extract features from EEG data. These features can then be used to help inform machine learning models how to classify different categories of data. 

Terminology: <br> 
- <b>Feature Extraction</b>: Ways of representing cleaned data in a succinct way for models to learn. 
- More terms defined in each feature extraction section

Experiment Setup: http://www.bbci.de/competition/iv/desc_2b.pdf <br>
General Overview of Feature Extraction for Motor Imagery: https://ieeexplore.ieee.org/abstract/document/8250265 <br>
PyEEG: http://pyeeg.sourceforge.net/ <br>
NeuroDSP: https://neurodsp-tools.github.io/neurodsp/index.html



## Imports

In [ ]:
from pathlib import Path # For making paths compatible on Windows and Macs

import pandas as pd # For working with DataFrames 
import numpy as np # For ease of array manipulation, stats, and some feature extraction
import matplotlib.pyplot as plt # For plotting pretty plots :) 
import scipy.signal as signal # For calculating PSDs and plotting spectrograms

import pyeeg # For pyeeg implemented features
from neurodsp.spectral import compute_spectrum # for smoothed PSD computation
from neurodsp.timefrequency import amp_by_time, freq_by_time, phase_by_time # For neurodsp features


## Constants

In [ ]:
eeg_fs = 250 # Data was recorded at 250 Hz
eeg_chans = ["C3", "Cz", "C4"] # 10-20 system 
eog_chans = ["EOG:ch01", "EOG:ch02", "EOG:ch03"] 
all_chans = eeg_chans + eog_chans
event_types = {0:"left", 1:"right"}

## Helper Functions

In [ ]:
# Multiple bar graph plotting
def plotMultipleBarGraphs(bars, bar_width, bar_names, group_names, error_values=None, title=None, xlabel=None, ylabel=None): 
    if len(bar_names) != len(bars):
        print("group names must be same length as bars")
        return 
    # Set position of bar on X axis
    positions = list()
    positions.append(np.arange(len(bars[0])))
    for i, bar in enumerate(bars): 
        if i>0: 
            positions.append([x + bar_width for x in positions[i-1]])

    # Make the plot
    for i, pos in enumerate(positions):
        plt.bar(pos, bars[i], width=bar_width, label=bar_names[i])
    
    if error_values is not None: 
        for i, pos in enumerate(positions):
            plt.errorbar(pos, bars[i], yerr=error_values[i], fmt='.k')
    
    # Add xticks on the middle of the group bars
    if xlabel: 
        plt.xlabel(xlabel)
    if ylabel: 
        plt.ylabel(ylabel)
    if title: 
        plt.title(title)
    plt.xticks([r + bar_width for r in range(len(bars[0]))], group_names)

    # Create legend & Show graphic
    plt.legend()
    plt.show()

## Load Epoched Data

In [ ]:
# These + epoched_test.pkl are the epochs that will be used in accuracy evaluation
epoch_df_filename = Path("./data/epoched_train.pkl")
eeg_epoch_full_df = pd.read_pickle(epoch_df_filename)
eeg_epoch_full_df.head(2)

## PyEEG Features

In [ ]:
# Power Bin extraction
import pyeeg
def getPowerRatio(eeg_data, binning, eeg_fs=250):
    power, power_ratio = pyeeg.bin_power(eeg_data, binning, eeg_fs)
    return np.array(power_ratio)
def getIntervals(binning): 
    intervals = list()
    for i, val in enumerate(binning[:-1]): 
        intervals.append((val, binning[i+1]))
    return intervals

### >> Power Bin
Power Bins are very widely used for EEG analysis to reduce PSDs into fewer features for Machine Learning. Spectral Power in the bin normalized by power in all spectral bins. 

In [ ]:
# Create a dataframe with the event_type and the power bin information for each trial

power_ratios = {'y': []}
binning=[0.5, 4, 7, 12, 30]
intervals = getIntervals(binning)
for i in range(0, len(eeg_epoch_full_df)): 
    event_type = eeg_epoch_full_df['event_type'][i]
    for ch in eeg_chans: 
        ratios = getPowerRatio(eeg_epoch_full_df[ch][i][:], binning)
        for j, interval in enumerate(intervals): 
            key = ch + "_" + str(interval)
            if key not in power_ratios: 
                power_ratios[key] = list()
            power_ratios[key].append(ratios[j])
    power_ratios['y'].append(eeg_epoch_full_df['event_type'][i])

power_ratios_df = pd.DataFrame(power_ratios)
power_ratios_df.head(2)

In [ ]:
# Calculate the standard error means between epochs for each channel from the power ratios obtained previously

chan_frequency_sems = {}
chan_frequency_avgs = {}

for event_type in event_types: 
    for ch in eeg_chans: 
        for interval in intervals: 
            key = ch + "_" + str(interval)
            if key not in chan_frequency_sems: 
                chan_frequency_sems[key] = list()
                chan_frequency_avgs[key] = list()
            this_data = power_ratios_df[power_ratios_df['y'] == event_type][key]
            sem = np.std(this_data) / np.sqrt(len(this_data)) # Standard Error of Mean calculation
            chan_frequency_sems[key].append(sem)
            chan_frequency_avgs[key].append(np.mean(this_data))
            
std_err_df = pd.DataFrame(chan_frequency_sems)
avg_df = pd.DataFrame(chan_frequency_avgs)

In [ ]:
# Plot average power ratios for each electrode

for chan in eeg_chans: 
    chan_of_interest = chan
    event_power_ratios = {}
    event_sems = {}
    power_ratios_for_chan = []
    sem_for_chan = []
    for event_type in event_types: 
        if event_type not in event_power_ratios: 
            event_power_ratios[event_type] = []
            event_sems[event_type] = []
        for interval in intervals: 
            key = chan_of_interest + "_" + str(interval)
            event_power_ratios[event_type].append(avg_df[key][event_type])
            event_sems[event_type].append(std_err_df[key][event_type])

    event_sems_df = pd.DataFrame(event_sems)
    event_power_ratios_df = pd.DataFrame(event_power_ratios)
    
    plt.title(chan_of_interest)
    plt.ylim((0, 0.5))
    plt.ylabel("Power Ratio")
    plotMultipleBarGraphs(np.transpose(np.array(event_power_ratios_df)), 0.15, [0, 1], intervals, error_values=np.transpose(np.array(event_sems_df)))
    
    

### >> Power Band Ratios
Band ratios can be helpful because they help characterize the ratio between different power bins within a trial. Here we provide getting theta / beta ratios, but you can try other combinations! 

In [ ]:
## Get band ratio features 

theta_beta_ratios = {} # Keys will be chan_theta_beta
# Iterate through rows of power_ratios_df
for i, row in power_ratios_df.iterrows(): 
    for ch in eeg_chans: 
        curr_key = ch + "_theta_beta"
        if curr_key not in theta_beta_ratios: 
            theta_beta_ratios[curr_key] = []
            
        # Calculate band ratios and append to dictionary
        power_bin_theta_key = ch + "_(4, 7)"
        power_bin_beta_key = ch + "_(12, 30)"
        theta_val = row[power_bin_theta_key]
        beta_val = row[power_bin_beta_key]
        theta_beta_ratios[curr_key].append(theta_val / beta_val)

# Create df for band ratios: band_ratios_df 
band_ratios_df = pd.DataFrame(theta_beta_ratios)
display(band_ratios_df.head(2))

# Concatenate power_ratios_df with band_ratios_df to get full feature df
feature_df = pd.concat([power_ratios_df, band_ratios_df], axis=1)
display(feature_df.head(2))

### Channel Relationships
Channel differences and ratios can provide spatial insights! 

In [ ]:
## Get channel ratio features

# Similar algorithm as Band Ratios, but for each interval
C3_C4_differences = {} 
for i, row in power_ratios_df.iterrows(): 
    for interval in intervals: 
        curr_key = "C3_C4_diff_" + str(interval)
        if curr_key not in C3_C4_differences: 
            C3_C4_differences[curr_key] = []
            
        # Calculate band ratios and append to dictionary
        power_bin_C3_key = "C3_" + str(interval)
        power_bin_C4_key = "C4_" + str(interval)
        
        C3_val = row[power_bin_C3_key]
        C4_val = row[power_bin_C4_key]
        
        C3_C4_differences[curr_key].append(C3_val - C4_val)

# Create df for band ratios: band_ratios_df 
C3_C4_differences_df = pd.DataFrame(C3_C4_differences)
display(C3_C4_differences_df.head(2))

# Concatenate power_ratios_df with band_ratios_df to get full feature df
feature_df = pd.concat([power_ratios_df, band_ratios_df, C3_C4_differences_df], axis=1)
display(feature_df.head(2))

### >> First and Second Difference average and max
Helpful for seeing how dramatically changing the recorded values are. 

In [ ]:
## Get first and second order diffs
# pyeeg.first_order_diff(X) for first order diffs
# Write your own second order diff algorithm! 

### >> Hjorth Parameters
Activity: How much high frequency there is (this is just variance) <br>
Mobility: Represents standard deviation of power spectrum <br>
Complexity: Similarity to pure sine wave <br> 


In [ ]:
## Get Hjorth Mobility and Complexity
# pyeeg.hjorth(X, D=None)

### >> Other Stats? 
What other features can you come up with? The posibilities in analyzing temporal vs frequency vs spatial information with respect to neural data is endless! :) Maybe you'll come up with the next best feature to distinguish these two classes and teach us something new about the brain! 

In [ ]:
## Experiment with finding new features using PyEEG, scipy, numpy! 

## NeuroDSP (i.e. Neuro Digital Signal Processing)
These are a subset of methods from the library that do a lot of fancy signal processing math to give you these time series features about the signal. Stats, derivitives, and time series analysis can be applied to this to help you extract even more features from the neural data. Coupling of these features (i.e. having these features synchronize in time) have been studied to represent cross-brain mental functions such as multimodal sensory processing, memory encoding/decoding, and motor activity!
<br><br>
~ If you want to learn more about how amplitude and phase coupling can be applied to Motor Imagery classification: <br>
<b>Amplitude and phase coupling measures for feature extraction in an EEG-based brain–computer interface.</b> Qingguo, W. et al (2007)  https://iopscience.iop.org/article/10.1088/1741-2560/4/2/012/pdf

~ NeuroDSP is a project from the Voytek Lab here at UCSD! <br>
Cole, S., Donoghue, T., Gao, R., & Voytek, B. (2019). <b>NeuroDSP: A package for
neural digital signal processing.</b> Journal of Open Source Software, 4(36), 1272.
DOI: 10.21105/joss.01272 https://doi.org/10.21105/joss.01272

In [ ]:
# Let's look at a single channel + trial, at alpha band
# Try changing this to different channels or signals :) 
sig = eeg_epoch_full_df['C3'][0]
alpha_range = (7, 12)

### >> Amplitude by time (instantaneous amplitude) 
This is defined as the power of the provided frequency band at a particular point in time. 

In [ ]:
# Calculate and plot alpha amplitude over time
amp = amp_by_time(sig, eeg_fs, alpha_range)
plt.figure(figsize=(15,2))
plt.plot(sig, label="EEG")
plt.plot(amp, label="amp over time")
plt.legend()
plt.title("Alpha power amplitude over time")
plt.show()

### >> Phase by time (instantaneous phase) 
This is defined as the phase of the signal over time. Phase is the angle of the sine wave that is represented by the frequency range. 

In [ ]:
pha = phase_by_time(sig, eeg_fs, alpha_range)
plt.figure(figsize=(15,2))
plt.plot(sig, label="EEG")
plt.plot(pha, label="phase over time")
plt.legend()
plt.title("Instantaneous phase over time")
plt.show()

### >> Frequency by time (instantaneous frequency)
This is defined as the temporal derivative of instantaneous phase (i.e. how much does the phase change). 

In [ ]:
i_f = freq_by_time(sig, eeg_fs, alpha_range)
plt.figure(figsize=(15,2))
plt.plot(sig, label="EEG")
plt.plot(i_f, label="freq over time")
plt.legend()
plt.title("Instantaneous frequency over time")
plt.show()

## Now you have many features to put into your Machine Learning algorithm or plot with statistics to find differences between the classes! 